In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import html5lib
import os

In [2]:
def createDf():
    data = {'round': [], 'season': [], 'date':[], 'hour':[], 'home team': [], 'guest team': [], 'score': [],'winners': [],'lastTime': []}
    df = pd.DataFrame(data)
    return df

In [3]:
def season(firstY,secondY):
    years = firstY+"-"+secondY
    return years

In [4]:
def load_soup_object(years,rounds):
    if years == "2016-2017":
        url = "https://www.worldfootball.net/schedule/esp-primera-division-"+years+"-spieltag_2/"+rounds
    else:
        url = "https://www.worldfootball.net/schedule/esp-primera-division-"+years+"-spieltag/"+rounds
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    return soup

In [5]:
def insertSoupToDataFrame(soup,df,rounds,years):
    
    date=list()
    hour=list()
    homeTeam=list()
    guestTeam=list()
    score=list()
    
    table = soup.find('table',attrs={"class":"standard_tabelle"})
    
    for row in table("tr"):
        cells = row("td")
        if(len(cells)<6):
            continue
        if cells[0].get_text() != "":
            temp = cells[0].get_text()
            date.append(cells[0].get_text())
        else:
            date.append(temp)
        hour.append(cells[1].get_text())
        homeTeam.append(cells[2].get_text().strip())
        guestTeam.append(cells[4].get_text().strip())
        score.append(cells[5].get_text().strip())
        
        
    data = {'round': rounds, 'season': years, 'date':date, 'hour':hour, 'home team': homeTeam, 'guest team': guestTeam, 'score': score}
    df = pd.DataFrame(data)
    return df
    

In [6]:
def getData():
    yearList = ["2000", "2001", "2002","2003", "2004", "2005","2006", "2007", "2008","2009",
                "2010", "2011","2012", "2013", "2014","2015", "2016", "2017","2018", "2019", "2020","2021", "2022"]

    roundsList = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
                  "11", "12","13", "14", "15", "16", "17", "18", "19", "20",
                  "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
                  "31", "32", "33", "34", "35", "36", "37", "38"]

    df = createDf()
    tempYear = 0;
    tempRounds = 0;
    for year in yearList:
        if yearList[tempYear] == "2022" :
            break
        years = season(yearList[tempYear],yearList[tempYear+1])
        for rounds in roundsList:
            soup = load_soup_object(years,rounds)
            df1 = insertSoupToDataFrame(soup,df,rounds,years)
            df = df.append(df1, ignore_index=True)
            tempRounds = tempRounds+1
        tempYear=tempYear+1
    
    return df

In [7]:
def getWinners(df):
    df1 = df.copy()
    temp = df.score
    winners = list()
    for i in temp:
        if i[0] < i[2]:
            winners.append("Away")
        elif i[0] > i[2]:
            winners.append("Home")
        else:
            winners.append("Draw")
    df1.winners = winners
    return df1

In [12]:
def getLastTimeWinners(df):
    df1 = df.copy()
    temp = df.score
    lastTiemWinners = list()
    for i in temp:
        try:
            if i[5] < i[7]:
                lastTiemWinners.append("Away")
            elif i[5] > i[7]:
                lastTiemWinners.append("Home")
            else:
                lastTiemWinners.append("Draw")
        except IndexError:
            lastTiemWinners.append("NaN")
    df1.lastTime = lastTiemWinners
    return df1

In [9]:
df = getData()

In [10]:
df

,round,season,date,hour,home team,guest team,score,winners,lastTime
0,1,2000-2001,09/09/2000,,Real Zaragoza,Espanyol Barcelona,1:2 (0:0),NaN,NaN
1,1,2000-2001,09/09/2000,,Real Madrid,Valencia CF,2:1 (0:0),NaN,NaN
2,1,2000-2001,09/09/2000,,FC Barcelona,Málaga CF,2:1 (2:0),NaN,NaN
3,1,2000-2001,09/09/2000,,Deportivo La Coruña,Athletic Bilbao,2:0 (0:0),NaN,NaN
4,1,2000-2001,09/09/2000,,Real Sociedad,Racing Santander,2:2 (0:0),NaN,NaN
...,...,...,...,...,...,...,...,...,...
8355,38,2021-2022,22/05/2022,17:30,Sevilla FC,Athletic Bilbao,-:-,NaN,NaN
8356,38,2021-2022,22/05/2022,17:30,Valencia CF,Celta Vigo,-:-,NaN,NaN
8357,38,2021-2022,22/05/2022,17:30,Rayo Vallecano,Levante UD,-:-,NaN,NaN
8358,38,2021-2022,22/05/2022,17:30,Real Madrid,Real Betis,-:-,NaN,NaN


In [14]:
df1 = getWinners(df)
df1 = getLastTimeWinners(df)

df1

,round,season,date,hour,home team,guest team,score,winners,lastTime
0,1,2000-2001,09/09/2000,,Real Zaragoza,Espanyol Barcelona,1:2 (0:0),NaN,Draw
1,1,2000-2001,09/09/2000,,Real Madrid,Valencia CF,2:1 (0:0),NaN,Draw
2,1,2000-2001,09/09/2000,,FC Barcelona,Málaga CF,2:1 (2:0),NaN,Home
3,1,2000-2001,09/09/2000,,Deportivo La Coruña,Athletic Bilbao,2:0 (0:0),NaN,Draw
4,1,2000-2001,09/09/2000,,Real Sociedad,Racing Santander,2:2 (0:0),NaN,Draw
...,...,...,...,...,...,...,...,...,...
8355,38,2021-2022,22/05/2022,17:30,Sevilla FC,Athletic Bilbao,-:-,NaN,NaN
8356,38,2021-2022,22/05/2022,17:30,Valencia CF,Celta Vigo,-:-,NaN,NaN
8357,38,2021-2022,22/05/2022,17:30,Rayo Vallecano,Levante UD,-:-,NaN,NaN
8358,38,2021-2022,22/05/2022,17:30,Real Madrid,Real Betis,-:-,NaN,NaN
